In [3]:
import asyncio
import os
import subprocess
from pydub import AudioSegment
import shutil
import uuid
import re
import whisper
from googletrans import Translator
from gtts import gTTS
from tensorflow.keras.models import Model, load_model
import pickle  # Added import for pickle
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences
import edge_tts
import numpy as np

from elevenlabs import ElevenLabs  # Import ElevenLabs client

from transformers import MarianTokenizer, MarianMTModel
import torch

C:\Users\15616\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\15616\anaconda3\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warnin

In [2]:
# # Initialize ElevenLabs client (replace with your API key)
# elevenlabs_client = ElevenLabs(api_key="sk_a931f6a0acdf587ebe3a0b227eeb5890f192dc27711cd832")

In [4]:
FEMALE_VOICES={'Vietnamese': 'vi-VN-HoaiMyNeural',
 'Bengali': 'bn-BD-NabanitaNeural',
 'Thai': 'th-TH-PremwadeeNeural',
 'English': 'en-AU-NatashaNeural',
 'Portuguese': 'pt-BR-FranciscaNeural',
 'Arabic': 'ar-AE-FatimaNeural',
 'Turkish': 'tr-TR-EmelNeural',
 'Spanish': 'es-AR-ElenaNeural',
 'Korean': 'ko-KR-SunHiNeural',
 'French': 'fr-BE-CharlineNeural',
 'Indonesian': 'id-ID-GadisNeural',
 'Russian': 'ru-RU-SvetlanaNeural',
 'Hindi': 'hi-IN-SwaraNeural',
 'Japanese': 'ja-JP-NanamiNeural',
 'Afrikaans': 'af-ZA-AdriNeural',
 'Amharic': 'am-ET-MekdesNeural',
 'Azerbaijani': 'az-AZ-BanuNeural',
 'Bulgarian': 'bg-BG-KalinaNeural',
 'Bosnian': 'bs-BA-VesnaNeural',
 'Catalan': 'ca-ES-JoanaNeural',
 'Czech': 'cs-CZ-VlastaNeural',
 'Welsh': 'cy-GB-NiaNeural',
 'Danish': 'da-DK-ChristelNeural',
 'German': 'de-AT-IngridNeural',
 'Greek': 'el-GR-AthinaNeural',
 'Irish': 'ga-IE-OrlaNeural',
 'Galician': 'gl-ES-SabelaNeural',
 'Gujarati': 'gu-IN-DhwaniNeural',
 'Hebrew': 'he-IL-HilaNeural',
 'Croatian': 'hr-HR-GabrijelaNeural',
 'Hungarian': 'hu-HU-NoemiNeural',
 'Icelandic': 'is-IS-GudrunNeural',
 'Italian': 'it-IT-ElsaNeural',
 'Javanese': 'jv-ID-SitiNeural',
 'Georgian': 'ka-GE-EkaNeural',
 'Kazakh': 'kk-KZ-AigulNeural',
 'Khmer': 'km-KH-SreymomNeural',
 'Kannada': 'kn-IN-SapnaNeural',
 'Lao': 'lo-LA-KeomanyNeural',
 'Lithuanian': 'lt-LT-OnaNeural',
 'Latvian': 'lv-LV-EveritaNeural',
 'Macedonian': 'mk-MK-MarijaNeural',
 'Malayalam': 'ml-IN-SobhanaNeural',
 'Mongolian': 'mn-MN-YesuiNeural',
 'Marathi': 'mr-IN-AarohiNeural',
 'Malay': 'ms-MY-YasminNeural',
 'Maltese': 'mt-MT-GraceNeural',
 'Burmese': 'my-MM-NilarNeural',
 'Norwegian Bokmål': 'nb-NO-PernilleNeural',
 'Nepali': 'ne-NP-HemkalaNeural',
 'Dutch': 'nl-BE-DenaNeural',
 'Polish': 'pl-PL-ZofiaNeural',
 'Pashto': 'ps-AF-LatifaNeural',
 'Romanian': 'ro-RO-AlinaNeural',
 'Sinhala': 'si-LK-ThiliniNeural',
 'Slovak': 'sk-SK-ViktoriaNeural',
 'Slovenian': 'sl-SI-PetraNeural',
 'Somali': 'so-SO-UbaxNeural',
 'Albanian': 'sq-AL-AnilaNeural',
 'Serbian': 'sr-RS-SophieNeural',
 'Sundanese': 'su-ID-TutiNeural',
 'Swedish': 'sv-SE-SofieNeural',
 'Swahili': 'sw-KE-ZuriNeural',
 'Tamil': 'ta-IN-PallaviNeural',
 'Telugu': 'te-IN-ShrutiNeural',
 'Chinese': 'zh-CN-XiaoxiaoNeural',
 'Ukrainian': 'uk-UA-PolinaNeural',
 'Urdu': 'ur-IN-GulNeural',
 'Uzbek': 'uz-UZ-MadinaNeural',
 'Zulu': 'zu-ZA-ThandoNeural'}



MALE_VOICES= {'Vietnamese': 'vi-VN-NamMinhNeural',
 'Bengali': 'bn-BD-PradeepNeural',
 'Thai': 'th-TH-NiwatNeural',
 'English': 'en-AU-WilliamNeural',
 'Portuguese': 'pt-BR-AntonioNeural',
 'Arabic': 'ar-AE-HamdanNeural',
 'Turkish': 'tr-TR-AhmetNeural',
 'Spanish': 'es-AR-TomasNeural',
 'Korean': 'ko-KR-HyunsuNeural',
 'French': 'fr-BE-GerardNeural',
 'Indonesian': 'id-ID-ArdiNeural',
 'Russian': 'ru-RU-DmitryNeural',
 'Hindi': 'hi-IN-MadhurNeural',
 'Japanese': 'ja-JP-KeitaNeural',
 'Afrikaans': 'af-ZA-WillemNeural',
 'Amharic': 'am-ET-AmehaNeural',
 'Azerbaijani': 'az-AZ-BabekNeural',
 'Bulgarian': 'bg-BG-BorislavNeural',
 'Bosnian': 'bs-BA-GoranNeural',
 'Catalan': 'ca-ES-EnricNeural',
 'Czech': 'cs-CZ-AntoninNeural',
 'Welsh': 'cy-GB-AledNeural',
 'Danish': 'da-DK-JeppeNeural',
 'German': 'de-AT-JonasNeural',
 'Greek': 'el-GR-NestorasNeural',
 'Irish': 'ga-IE-ColmNeural',
 'Galician': 'gl-ES-RoiNeural',
 'Gujarati': 'gu-IN-NiranjanNeural',
 'Hebrew': 'he-IL-AvriNeural',
 'Croatian': 'hr-HR-SreckoNeural',
 'Hungarian': 'hu-HU-TamasNeural',
 'Icelandic': 'is-IS-GunnarNeural',
 'Italian': 'it-IT-DiegoNeural',
 'Javanese': 'jv-ID-DimasNeural',
 'Georgian': 'ka-GE-GiorgiNeural',
 'Kazakh': 'kk-KZ-DauletNeural',
 'Khmer': 'km-KH-PisethNeural',
 'Kannada': 'kn-IN-GaganNeural',
 'Lao': 'lo-LA-ChanthavongNeural',
 'Lithuanian': 'lt-LT-LeonasNeural',
 'Latvian': 'lv-LV-NilsNeural',
 'Macedonian': 'mk-MK-AleksandarNeural',
 'Malayalam': 'ml-IN-MidhunNeural',
 'Mongolian': 'mn-MN-BataaNeural',
 'Marathi': 'mr-IN-ManoharNeural',
 'Malay': 'ms-MY-OsmanNeural',
 'Maltese': 'mt-MT-JosephNeural',
 'Burmese': 'my-MM-ThihaNeural',
 'Norwegian Bokmål': 'nb-NO-FinnNeural',
 'Nepali': 'ne-NP-SagarNeural',
 'Dutch': 'nl-BE-ArnaudNeural',
 'Polish': 'pl-PL-MarekNeural',
 'Pashto': 'ps-AF-GulNawazNeural',
 'Romanian': 'ro-RO-EmilNeural',
 'Sinhala': 'si-LK-SameeraNeural',
 'Slovak': 'sk-SK-LukasNeural',
 'Slovenian': 'sl-SI-RokNeural',
 'Somali': 'so-SO-MuuseNeural',
 'Albanian': 'sq-AL-IlirNeural',
 'Serbian': 'sr-RS-NicholasNeural',
 'Sundanese': 'su-ID-JajangNeural',
 'Swedish': 'sv-SE-MattiasNeural',
 'Swahili': 'sw-KE-RafikiNeural',
 'Tamil': 'ta-IN-ValluvarNeural',
 'Telugu': 'te-IN-MohanNeural',
 'Chinese': 'zh-CN-YunjianNeural',
 'Ukrainian': 'uk-UA-OstapNeural',
 'Urdu': 'ur-IN-SalmanNeural',
 'Uzbek': 'uz-UZ-SardorNeural',
 'Zulu': 'zu-ZA-ThembaNeural'}


LANGUAGES = {
    "Afrikaans": "af",
    "Amharic": "am",
    "Arabic": "ar",
    "Azerbaijani": "az",
    "Bulgarian": "bg",
    "Bengali": "bn",
    "Bosnian": "bs",
    "Catalan": "ca",
    "Czech": "cs",
    "Welsh": "cy",
    "Danish": "da",
    "German": "de",
    "Greek": "el",
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "Irish": "ga",
    "Galician": "gl",
    "Gujarati": "gu",
    "Hebrew": "he",
    "Hindi": "hi",
    "Croatian": "hr",
    "Hungarian": "hu",
    "Indonesian": "id",
    "Icelandic": "is",
    "Italian": "it",
    "Japanese": "ja",
    "Javanese": "jv",
    "Georgian": "ka",
    "Kazakh": "kk",
    "Khmer": "km",
    "Kannada": "kn",
    "Korean": "ko",
    "Lao": "lo",
    "Lithuanian": "lt",
    "Latvian": "lv",
    "Macedonian": "mk",
    "Malayalam": "ml",
    "Mongolian": "mn",
    "Marathi": "mr",
    "Malay": "ms",
    "Maltese": "mt",
    "Burmese": "my",
    "Norwegian Bokmål": "nb",
    "Nepali": "ne",
    "Dutch": "nl",
    "Polish": "pl",
    "Pashto": "ps",
    "Portuguese": "pt",
    "Romanian": "ro",
    "Russian": "ru",
    "Sinhala": "si",
    "Slovak": "sk",
    "Slovenian": "sl",
    "Somali": "so",
    "Albanian": "sq",
    "Serbian": "sr",
    "Sundanese": "su",
    "Swedish": "sv",
    "Swahili": "sw",
    "Tamil": "ta",
    "Telugu": "te",
    "Thai": "th",
    "Turkish": "tr",
    "Ukrainian": "uk",
    "Urdu": "ur",
    "Uzbek": "uz",
    "Vietnamese": "vi",
    "Chinese": "zh",
    "Zulu": "zu"
}


In [7]:
class AudioTranscriber:
    """Handles audio transcription using Whisper."""
    def __init__(self, model_name="base"):
        self.model = whisper.load_model(model_name)

    def transcribe(self, audio_filepath, output_txt_path, output_srt_path):
        if not os.path.exists(audio_filepath):
            raise FileNotFoundError(f"Audio file not found: {audio_filepath}")
        
        result = self.model.transcribe(audio_filepath, word_timestamps=True, fp16=False)
        
        os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(result["text"])
        
        os.makedirs(os.path.dirname(output_srt_path), exist_ok=True)
        with open(output_srt_path, 'w', encoding='utf-8') as f:
            for i, segment in enumerate(result["segments"], 1):
                start = segment["start"]
                end = segment["end"]
                text = segment["text"].strip()
                f.write(f"{i}\n")
                f.write(f"{self._format_srt_timestamp(start)} --> {self._format_srt_timestamp(end)}\n")
                f.write(f"{text}\n\n")
        
        print(f"Transcription saved at: {output_txt_path}")
        print(f"Original SRT saved at: {output_srt_path}")
        return output_srt_path, result["text"]

    @staticmethod
    def _format_srt_timestamp(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds % 1) * 1000)
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

In [10]:
class TextToSpeech:
    """Handles text-to-speech conversion with multi-language support."""
    def __init__(self, speed=1):
        self.speed = speed

    async def convert(self, text, output_path, language, gender, should_translate=False):
        target_language = LANGUAGES.get(language, "en")
        input_text = await self._translate_text(text, language) if should_translate else text
        print(f"Translated to {language}: {input_text}" if should_translate else f"Processing: {input_text}")
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Choose voice based on gender
        voice = FEMALE_VOICES.get(language, "en-AU-NatashaNeural") if gender == "Female" else MALE_VOICES.get(language, "en-AU-WilliamNeural")
        
        # Use edge-tts for audio generation
        tts = edge_tts.Communicate(text=input_text, voice=voice)
        await tts.save(output_path)
        
        audio = AudioSegment.from_file(output_path)
        if self.speed != 1:
            audio = self._adjust_speed(audio, self.speed)
        audio.export(output_path, format="mp3")
        print(f"Generated audio: {output_path}, Duration: {audio.duration_seconds}s")
        return audio

    async def _translate_text(self, text, target_language):
        """Translate text based on the target language using appropriate model."""
        if target_language == "Turkish":
            return await self._loadTurkishModelAndTranslate(text)
        elif target_language in ["Italian", "Spanish", "Dutch"]:
            model_paths = {
                "Italian": "transformer_en_ita_model",
                "Spanish": "transformer_en_es_model",
                "Dutch": "transformer_en_deu_model"
            }
            return self._load_and_translate(model_paths[target_language], text)
        else:
            raise ValueError(f"Unsupported language for translation: {target_language}")

    async def _loadTurkishModelAndTranslate(self, test_sentence):
        """Load Turkish Seq2Seq model and translate the input sentence."""
        output_dir = 'seq2seq'
        with open(f'{output_dir}/tokenizer/input_tokenizer.pkl', 'rb') as f:
            loaded_input_tokenizer = pickle.load(f)
        with open(f'{output_dir}/tokenizer/target_tokenizer.pkl', 'rb') as f:
            loaded_target_tokenizer = pickle.load(f)
        with open(f'{output_dir}/tokenizer/maxlen_values.pkl', 'rb') as f:
            maxlen = pickle.load(f)
        
        loaded_encoder_model = load_model(output_dir + '/encoder_model.h5', compile=False)
        loaded_decoder_model = load_model(output_dir + '/decoder_model.h5', compile=False)
        
        translation = self.translate_sentence(test_sentence, loaded_encoder_model, loaded_decoder_model, 
                                             loaded_input_tokenizer, loaded_target_tokenizer, 
                                             maxlen['input_maxlen'], maxlen['target_maxlen'])
        print(f"Input Sentence: {test_sentence}")
        print(f"Translated Sentence: {translation}")
        return translation

    def translate_sentence(self, sentence, encoder_model, decoder_model, input_tokenizer, target_tokenizer, input_maxlen, target_maxlen):
        """Translate a sentence using the loaded Seq2Seq model."""
        input_seq = input_tokenizer.texts_to_sequences([sentence])
        input_seq = pad_sequences(input_seq, maxlen=input_maxlen, padding='post')
        states_value = encoder_model.predict(input_seq, verbose=0)
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = target_tokenizer.word_index['<start>']
        stop_condition = False
        decoded_sentence = ''
        end_token_idx = target_tokenizer.word_index['<end>']

        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_word = target_tokenizer.index_word.get(sampled_token_index, '')
            if sampled_word == '<end>' or len(decoded_sentence.split()) >= target_maxlen:
                stop_condition = True
            elif sampled_word != '<start>':
                decoded_sentence += sampled_word + ' '
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            states_value = [h, c]
        
        return decoded_sentence.strip()

    def _load_and_translate(self, model_path, text, max_length=128, device=None):
        """Load MarianMT model and translate text."""
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"Loading MarianMT model from {model_path} for translation...")
        tokenizer = MarianTokenizer.from_pretrained(model_path)
        model = MarianMTModel.from_pretrained(model_path)
        model.to(device)
        model.eval()
        
        print(f"Tokenizing input text: '{text}'")
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length).to(device)
        
        with torch.no_grad():
            translated = model.generate(**inputs)
        
        translation = tokenizer.decode(translated[0], skip_special_tokens=True)
        print(f"Translated '{text}' to '{translation}'")
        return translation

    def _adjust_speed(self, audio, speed):
        """Adjust playback speed using pydub."""
        new_frame_rate = int(audio.frame_rate * speed)
        return audio._spawn(audio.raw_data, overrides={'frame_rate': new_frame_rate}).set_frame_rate(audio.frame_rate)

In [12]:
class SubtitleDubber:
    """Handles subtitle dubbing and SRT translation."""
    def __init__(self, transcriber: AudioTranscriber, tts: TextToSpeech):
        self.transcriber = transcriber
        self.tts = tts

    async def dub_srt(self, srt_file_path, dub_save_path, language="Japanese", gender="Male"):
        """Dub SRT file into audio and generate translated SRT."""
        subtitle_data = self._parse_srt_file(srt_file_path)
        new_folder_path = self._create_directory_for_srt(srt_file_path)
        audio_files_to_merge = []

        for subtitle in subtitle_data:
            text = subtitle['text']
            actual_duration = subtitle['end_time'] - subtitle['start_time']
            pause_time = subtitle['pause_time']
            silence_path = f"{new_folder_path}/{subtitle['previous_pause']}"
            self._create_silence(pause_time, silence_path)
            audio_files_to_merge.append(silence_path)
            audio_path = f"{new_folder_path}/{subtitle['audio_name']}"
            await self._convert_text_to_speech(text, audio_path, language, gender, actual_duration)
            audio_files_to_merge.append(audio_path)

        await self._merge_audio_files(audio_files_to_merge, dub_save_path)
        translated_srt_path = os.path.splitext(dub_save_path)[0] + "_translated.srt"
        await self._generate_translated_srt(subtitle_data, language, translated_srt_path)

    async def _convert_text_to_speech(self, text, audio_output_path, language, gender, actual_duration):
        temp_filename = "D:/Work/Faculty/Masters/NLP/project/content/temp_audio1.mp3"
        os.makedirs(os.path.dirname(temp_filename), exist_ok=True)
        audio = await self.tts.convert(text, temp_filename, language, gender, should_translate=True)
        
        if audio is None:
            print(f"Skipping {text} due to invalid or missing audio file: {temp_filename}")
            silence = AudioSegment.silent(duration=actual_duration)
            silence.export(audio_output_path, format="mp3")
            return

        tts_duration = len(audio)
        print(f"Generated temp audio duration: {tts_duration/1000}s, Expected: {actual_duration/1000}s")

        if actual_duration == 0:
            shutil.move(temp_filename, audio_output_path)
            return

        if tts_duration > actual_duration:
            speedup_factor = tts_duration / actual_duration
            speedup_filename = "D:/Work/Faculty/Masters/NLP/project/content/sped_up_audio.mp3"
            subprocess.run([
                "ffmpeg", "-i", temp_filename, "-filter:a", f"atempo={speedup_factor}", "-f", "mp3", speedup_filename
            ], check=True)
            shutil.move(speedup_filename, audio_output_path)
        elif tts_duration < actual_duration:
            silence_gap = actual_duration - tts_duration
            silence = AudioSegment.silent(duration=int(silence_gap))
            new_audio = audio + silence
            new_audio.export(audio_output_path, format="mp3")
        else:
            shutil.move(temp_filename, audio_output_path)

    @staticmethod
    def _create_silence(pause_duration, silence_file_path):
        min_duration = max(pause_duration, 100)
        silence = AudioSegment.silent(duration=min_duration)
        silence.export(silence_file_path, format="mp3")
        audio = AudioSegment.from_file(silence_file_path)
        print(f"Created silence file: {silence_file_path}, Duration: {audio.duration_seconds}s")
        return silence_file_path

    @staticmethod
    def _create_directory_for_srt(srt_file_path):
        srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
        random_uuid = str(uuid.uuid4())[:4]
        base_directory = "D:/Work/Faculty/Masters/NLP/project/content/dummy"
        os.makedirs(base_directory, exist_ok=True)
        new_directory = os.path.join(base_directory, f"{srt_base_name}_{random_uuid}")
        os.makedirs(new_directory, exist_ok=True)
        return new_directory

    @staticmethod
    async def _merge_audio_files(audio_paths, output_path):
        merged_audio = AudioSegment.silent(duration=0)
        for audio_path in audio_paths:
            print(f"Merging: {audio_path}")
            audio = AudioSegment.from_file(audio_path)
            merged_audio += audio
        merged_audio.export(output_path, format="mp3")

    async def _generate_translated_srt(self, subtitle_data, language, srt_output_path):
        translated_entries = []
        for entry in subtitle_data:
            # translated_text = await self.tts._translate(entry['text'], language)  # Use tts._translate()
            translated_text = await self.tts._loadTurkishModelAndTranslate(entry['text'])
            translated_entries.append({
                'entry_number': entry['entry_number'],
                'start_time': entry['start_time'],
                'end_time': entry['end_time'],
                'text': translated_text
            })
        
        with open(srt_output_path, 'w', encoding='utf-8') as f:
            for entry in translated_entries:
                f.write(f"{entry['entry_number']}\n")
                f.write(f"{self._format_srt_timestamp(entry['start_time'])} --> {self._format_srt_timestamp(entry['end_time'])}\n")
                f.write(f"{entry['text']}\n\n")
        print(f"Translated SRT saved at: {srt_output_path}")

    @staticmethod
    def _parse_srt_file(file_path):
        subtitle_entries = []
        default_start_time = 0
        previous_end_time = default_start_time
        entry_count = 1
        audio_name_format = "{}.mp3"
        pause_name_format = "{}_before_pause.mp3"

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            for i in range(0, len(lines), 4):
                time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
                start_time = SubtitleDubber._convert_to_milliseconds(time_info[0][0])
                end_time = SubtitleDubber._convert_to_milliseconds(time_info[0][1])

                current_entry = {
                    'entry_number': entry_count,
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': lines[i + 2].strip(),
                    'pause_time': start_time - previous_end_time if entry_count != 1 else start_time - default_start_time,
                    'audio_name': audio_name_format.format(entry_count),
                    'previous_pause': pause_name_format.format(entry_count),
                }

                subtitle_entries.append(current_entry)
                previous_end_time = end_time
                entry_count += 1

        return subtitle_entries

    @staticmethod
    def _convert_to_milliseconds(time_str):
        hours, minutes, second_millisecond = time_str.split(':')
        seconds, milliseconds = second_millisecond.split(",")
        return (
            int(hours) * 3600000 +
            int(minutes) * 60000 +
            int(seconds) * 1000 +
            int(milliseconds)
        )

    @staticmethod
    def _format_srt_timestamp(milliseconds):
        seconds = milliseconds / 1000
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds % 1) * 1000)
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

In [15]:
class VideoDubber:
    """Handles video dubbing with FFmpeg."""
    @staticmethod
    async def dub_video(original_video_path, dubbed_audio_path, output_video_path):
        """Replace original audio with dubbed audio in the video."""
        os.makedirs(os.path.dirname(output_video_path), exist_ok=True)
        ffmpeg_command = [
            "ffmpeg",
            "-i", original_video_path,
            "-i", dubbed_audio_path,
            "-c:v", "copy",
            "-map", "0:v:0",
            "-map", "1:a:0",
            "-shortest",
            "-y",
            output_video_path
        ]
        result = subprocess.run(ffmpeg_command, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Dubbed video saved at: {output_video_path}")
            return True
        else:
            print(f"Failed to generate video: {result.stderr}")
            return False

In [18]:
async def main(language="Turkish", gender="Male"):
    audio_file_path = "D:/Work/Faculty/Masters/NLP/project/Video_Dubbing/input files/trump_speech.mp4"
    output_video_path = "D:/Work/Faculty/Masters/NLP/project/Video_Dubbing/content/dubbed_video.mp4"
    dub_save_path = "D:/Work/Faculty/Masters/NLP/project/Video_Dubbing/content/dubbed_audio.mp3"
    txt_save_path = "D:/Work/Faculty/Masters/NLP/project/Video_Dubbing/content/transcription_output.txt"
    srt_save_path = "D:/Work/Faculty/Masters/NLP/project/Video_Dubbing/content/original_subtitles.srt"

    valid_languages = ["Turkish", "Italian", "Spanish", "Dutch"]
    valid_genders = ["Male", "Female"]
    
    if language not in valid_languages:
        raise ValueError(f"Invalid language: {language}. Supported languages: {valid_languages}")
    if gender not in valid_genders:
        raise ValueError(f"Invalid gender: {gender}. Use 'Male' or 'Female'.")

    transcriber = AudioTranscriber()
    tts = TextToSpeech()
    dubber = SubtitleDubber(transcriber, tts)
    video_dubber = VideoDubber()

    srt_file_path, _ = transcriber.transcribe(audio_file_path, txt_save_path, srt_save_path)
    await dubber.dub_srt(srt_file_path, dub_save_path, language, gender)
    await video_dubber.dub_video(audio_file_path, dub_save_path, output_video_path)

# Run with different languages and genders
if __name__ == "__main__":
    # Example 1: Turkish with Male voice (Seq2Seq)
    await main(language="Dutch", gender="Male")

    # Example 2: Spanish with Female voice (MarianMT)
    # asyncio.run(main(language="Spanish", gender="Female"))

    # Example 3: Italian with Male voice (MarianMT)
    # asyncio.run(main(language="Italian", gender="Male"))

    # Example 4: Dutch with Female voice (MarianMT)
    # asyncio.run(main(language="Dutch", gender="Female"))

Transcription saved at: D:/Work/Faculty/Masters/NLP/project/content/transcription_output.txt
Original SRT saved at: D:/Work/Faculty/Masters/NLP/project/content/original_subtitles.srt
Created silence file: D:/Work/Faculty/Masters/NLP/project/content/dummy\original_subtitles_2ce5/1_before_pause.mp3, Duration: 0.09995464852607709s
Loading MarianMT model from transformer_en_deu_model for translation...


C:\Users\15616\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Tokenizing input text: 'But while we are fighting every day to build up our nation,'
Translated 'But while we are fighting every day to build up our nation,' to 'aber während wir jeden tag um den bau unserer nation kämpfen'
Translated to Dutch: aber während wir jeden tag um den bau unserer nation kämpfen
Generated audio: D:/Work/Faculty/Masters/NLP/project/content/temp_audio1.mp3, Duration: 4.416s
Generated temp audio duration: 4.416s, Expected: 4.059s
Created silence file: D:/Work/Faculty/Masters/NLP/project/content/dummy\original_subtitles_2ce5/2_before_pause.mp3, Duration: 0.22095238095238096s
Loading MarianMT model from transformer_en_deu_model for translation...
Tokenizing input text: 'the far left only wants to wreck ruin and destroy our nation.'
Translated 'the far left only wants to wreck ruin and destroy our nation.' to 'die übrige Seite will nur noch ruinieren und unsere nation zerstören.'
Translated to Dutch: die übrige Seite will nur noch ruinieren und unsere nation zerstör